In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [2]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x.lower()))
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])

Ponieważ pusta wartość jest interpretowana jako float, należy ograniczyć zbiór.

In [3]:
dane = dane[dane['full_name'].apply(lambda x:isinstance(x, str))]

In [4]:
dane = dane.reset_index()

In [5]:
dane = dane.drop('index', axis=1)

In [4]:
dane.head(3)

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
0,0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us...",16.0,13.0,wymiar sprawiedliwości,przepisy
1,1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb...",14.0,16.0,ustrój państwa,ustrój państwa
2,2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze...",14.0,0.0,ustrój państwa,polityka


In [62]:
dane[dane['party_final']=='Platforma Obywatelska']

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
158310,158869,31-08-2011,Panie Przewodniczący! Z należnym szacunkiem p...,Jan Kulas,Platforma Obywatelska,"[przewodniczący, należny, szacunek, przyjrzeć,...",16.0,13.0,wymiar sprawiedliwości,przepisy
158311,158870,31-08-2011,Moje oświadczenie poselskie poświęcam działaln...,Jan Kulas,Platforma Obywatelska,"[oświadczenie, poselski, poświęcać, działalnoś...",19.0,14.0,patriotyzm,wojsko
158312,158871,31-08-2011,"Wyobraźmy sobie, że gdzieś w tzw. cywilizowan...",Paweł Graś,Platforma Obywatelska,"[wyobrazić, siebie, cywilizować, świat, kraj, ...",7.0,0.0,NaN,polityka
158313,158872,31-08-2011,"Doszło do skandalicznego głosowania, albowiem...",Paweł Graś,Platforma Obywatelska,"[dojść, skandaliczny, głosować, albowiem, pose...",14.0,0.0,ustrój państwa,polityka
158314,158873,31-08-2011,Z głębokim smutkiem i wielkim zażenowaniem pr...,Paweł Graś,Platforma Obywatelska,"[głęboki, smutek, wielki, zażenowanie, przyjąć...",33.0,0.0,polska,polityka


Utworzenie słownika autor - dokument

## Przygotowanie słowników itp.

In [13]:
dane.head()

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
0,0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us...",16.0,13.0,wymiar sprawiedliwości,przepisy
1,1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb...",14.0,16.0,ustrój państwa,ustrój państwa
2,2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze...",14.0,0.0,ustrój państwa,polityka
3,3,01-02-1992,"Wszystkim nam bardzo zależy na tym, aby urato...",Alojzy Szablewski,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,"[wszystek, zależeć, uratować, przemysł, stocze...",10.0,5.0,NaN,przedsiębiorstwa
4,4,01-02-1992,Z brzmienia pkt. 1 projektu uchwały odczytuję...,Andrzej Baraniecki,Sojusz Lewicy Demokratycznej,"[brzmię, punkt, projekt, uchwała, odczytować, ...",10.0,16.0,NaN,ustrój państwa


In [14]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane.index):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)

## Słowniki i dictionary po utworzeniu są zrzucone do pickli...

In [15]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane.index):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)
doc2aut = dane['full_name'].T.to_dict()
corp = dane['speech_rawest']
bigram = gensim.models.Phrases(corp, min_count=20, threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
corp = [bigram_mod[doc] for doc in corp]
dictionary = gensim.corpora.Dictionary(corp)
dictionary.filter_extremes(no_above=0.2, no_below=50)
dictionary.compactify()
pickle.dump(dictionary, open('dict_atm.p', 'wb'))
BOW = [dictionary.doc2bow(text) for text in corp]
pickle.dump(BOW, open('bow_atm.p', 'wb'))

In [16]:
from gensim.corpora import mmcorpus
from gensim.test.utils import common_dictionary, datapath, temporary_file
print(mmcorpus.MmCorpus(datapath('testcorpus.mm'))[8])

[(4, 1.0), (10, 1.0), (11, 1.0)]


In [42]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=BOW, num_topics=30, id2word=dictionary.id2token, 
                               author2doc=aut2doc, passes=5, eval_every=0, iterations=1)

Wall time: 10min 50s


In [44]:
pickle.dump(model, open('model_atm.p', 'wb'))

## ..... i ładowane

In [5]:
model = pickle.load(open('model_atm.p', 'rb'))
dictionary = pickle.load(open('dict_atm.p', 'rb'))
BOW = pickle.load(open('bow_atm.p', 'rb'))

## Opis tematów

In [43]:
for i in range(30):
    print('TOPIC ', i)
    for a, b in model.show_topic(i):
        print(a)

TOPIC  0
mieszkanie
mieszkaniowy
spółdzielnia
kredyt
własność
nieruchomość
gmina
spółdzielczy
budownictwo
koszt
TOPIC  1
międzynarodowy
konwencja
rzeczypospolity
klub_parlamentarny
umowa
obywatel
gospodarczy
zagraniczny
stanowić
partia
TOPIC  2
służba
administracja
wojskowy
wewnętrzny
policja
bezpieczeństwo
ochrona
cywilny
zapis
informacja
TOPIC  3
polak
żołnierz
naród
rzeczypospolity
prezydent
historia
armia
władza
wojna
życie
TOPIC  4
myśleć
wielki
debata
solidarność
program
rada
krajowy
budżet
telewizja
szczególnie
TOPIC  5
rodzina
świadczenie
fundusz
niepełnosprawny
pracownik
dziecko
pomoc
ubezpieczenie
polityka
złoty
TOPIC  6
unia_europejski
polityka
europejski
naród
telewizja
medium
rodzina
europa
polak
rada
TOPIC  7
dziecko
szkoła
nauczyciel
edukacja
szkół
nauka
rodzic
oświata
rodzina
młodzież
TOPIC  8
szanowny
naprawdę
sprawiedliwość
polak
prosty
oczywiście
pieniądz
premier
pis
zrobić
TOPIC  9
prywatyzacja
przedsiębiorstwo
gospodarka
bank
gospodarczy
skarb
górnictwo
program
pań

In [67]:
topics = ['rynek mieszkaniowy', 'umowy gospodarcze', 'służby państwowe', 'naród - patos', '4', 'świadczenia pomocowe', 
          'europa', 'edukacja', '8', 'gospodarka', 'UE', 'sądownictwo', 'służba zdrowia', 'konstytucja', 'wybory', 'posiedzenie'
         , 'budżet', 'rolnictwo', 'ochrona środowiska', 'wymiar prawa', 'dyskusja', 'polityka', 'finanse', 'przedsiębiorstwa',
         'samorządy', 'miasta', 'drogi', 'ustrój państwa', '28', 'rząd']

Tematy są trudne do opisania. Być może mówców jest zbyt dużo i przez to się rozdrabnia?
Później zweryfikujemy czy lepiej wychodzi dla partii.

In [64]:
pd.DataFrame(model.get_author_topics('Elżbieta Witek'))

,0,1
0,1,0.010174
1,2,0.014873
2,3,0.024079
3,5,0.105116
4,6,0.018230
5,7,0.223540
6,8,0.171279
7,11,0.022124
8,12,0.018382
9,13,0.022877


In [66]:
pd.DataFrame(topics)

,0
0,rynek mieszkaniowy
1,polityka zagraniczna
2,służby państwowe
3,naród - patos
4,4
5,świadczenia pomocowe
6,europa
7,edukacja
8,8
9,gospodarka


## Model dla partii

In [9]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['party_final'], dane.index):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)
doc2aut = dane['party_final'].T.to_dict()

Model wygenerowany na parametrach takich jak poprzednio był bardzo trudny do opisu.
Spróbuję innych wersji - z mniejsza liczbą tematów, albo większą liczbą iteracji.

In [25]:
dictionary.id2token

{0: 'agent',
 1: 'ażeby',
 2: 'bezprecensnsowy',
 3: 'bezzasadny',
 4: 'budżetowy',
 5: 'celowo',
 6: 'cierpieć',
 7: 'ciągle',
 8: 'cywilny',
 9: 'czasie',
 10: 'czyn',
 11: 'czyn_przestępczy',
 12: 'dawno',
 13: 'decyzja',
 14: 'dolegliwość',
 15: 'dostać',
 16: 'dosyć',
 17: 'drastyczny',
 18: 'dyplomat',
 19: 'długo',
 20: 'echo',
 21: 'egzekwować',
 22: 'furtka',
 23: 'generalnie',
 24: 'granica',
 25: 'głos',
 26: 'głosować',
 27: 'historia',
 28: 'iluzja',
 29: 'inicjatywa',
 30: 'intencja',
 31: 'internowany',
 32: 'istnieć',
 33: 'istotny',
 34: 'itpa',
 35: 'kara',
 36: 'karny',
 37: 'kierować',
 38: 'kodeks_karny',
 39: 'komorniczy',
 40: 'konieczny',
 41: 'konsekwencja',
 42: 'konstrukcja',
 43: 'krzywdzić',
 44: 'kukliński',
 45: 'kwartał',
 46: 'kwestionowanie',
 47: 'legalność',
 48: 'legalny',
 49: 'lekko',
 50: 'mld_złoty',
 51: 'mygólnie',
 52: 'myśleć',
 53: 'nadmierny',
 54: 'nadzwyczajny',
 55: 'naruszenie',
 56: 'następca',
 57: 'niebezpieczny',
 58: 'niekiedy',
 

In [34]:
from gensim.models import AuthorTopicModel
%time model1 = AuthorTopicModel(corpus=BOW, num_topics=20, id2word=dictionary.id2token, author2doc=aut2doc, passes=3, eval_every=0, iterations=1)

Wall time: 6min


In [27]:
pickle.dump(model1, open('model_ptm_1.p', 'wb'))

In [8]:
dictionary = pickle.load(open('dict_atm.p', 'rb'))
BOW = pickle.load(open('bow_atm.p', 'rb'))
model1 = pickle.load(open('model_ptm_1.p', 'rb'))

EOFError: Ran out of input

In [35]:
for i in range(20):
    print('TOPIC ', i)
    for a, b in model1.show_topic(i):
        print(a)

TOPIC  0
unia
klub_parlamentarny
głosować
sprawozdanie
zapis
arta
senat
gospodarczy
zdanie
działalność
TOPIC  1
wydawać
budżet
unia
wolność
system
polityczny
sądzić
konstytucja
gmina
polityka
TOPIC  2
demokratyczny
sojusz_lewica
arta
gospodarczy
budżet
usta
zapis
polityka
rada
działalność
TOPIC  3
naprawdę
dziecko
szanowny
budżet
sprawiedliwość
system
obywatel
polak
pieniądz
podatek
TOPIC  4
wolność
gospodarczy
sąd
nauka
arta
własność
państwowy
władza
unia
szkolnictwo
TOPIC  5
dziecko
życie
rodzina
kościół
telewizja
polityka
kobieta
polityczny
ochrona
wobec
TOPIC  6
bank
budżet
rolnictwo
rynek
rolny
gospodarczy
fundusz
rolnik
produkcja
kredyt
TOPIC  7
narodowy
ruch_katolicki
samorząd
pięknie
polak
gmina
samorząd_terytorialny
finansowy
środowisko
koło
TOPIC  8
prezydent
rada
arta
solidarność
reforma
usta
konstytucja
zdanie
gmina
konfederacja
TOPIC  9
demokracja
system
zdrowie
oczywiście
społeczeństwo
polityczny
uważać
popierać
zarówno
szczególność
TOPIC  10
unia_europejski
rolnik
polak


In [ ]:
topics_p = ['narodowy', 'polityka', 'budżet', '3', '4', 'górnictwo', '']